### ARTIFICIAL INTELLIGENCE FUNDAMENTALS PROJECT 2024/2025 - Charisardo Team 

[Github repository](https://github.com/sypeeoui/Charisardo) 

#### 1. Introduction
Our project focused on the design and implementation of several agent algorithms capable of engaging in Pokémon battles with randomly selected teams, determining the best move from those available. 
The task was part of the battle track of the VGC AI Competition 2024.
We simulated the battles using the VGC AI Framework, which we modified to introduce non-determinism and to add additional features to the Pokémon. <br>
We developed four strategies using three approaches:
* Heuristic-based method
* Minimax search with alpha-beta pruning
* Greedy search (two versions)

#### 2. Related works
To build the Pokémon teams and simulate the battles we used [VGC AI framework](https://gitlab.com/DracoStriker/pokemon-vgc-engine), an environment designed to emulate the scenario of human video game championships of Pokémon with AI agents, including the game balance aspect. 
We also used the policies already implemented in the framework as a benchmarck to evaluate our algorithms. More information about the VCG AI framework are avaiable in the related [paper](https://ieeexplore.ieee.org/document/9618985). <br>
To design and implement some of our algorithms we referred to the textbook *Artificial Intelligence: a Modern Approach*, by Russell and Norvig, in particular chapters 3.5, 5.2 and 5.3.



#### 3. Methodologies
spiegate modifiche a environment e algoritmi
#### 3.1 Environment

#### 3.2 Algorithms

##### 3.2.1 Heuristical

##### 3.2.2 PrunedTreeSearch

##### 3.2.3 WeightedGreedy 1/2

#### 4. Assessment
We evaluated the performance of our algorithms comparing them to each other and to six already implemented policies: 'RandomPlayer', 'TypeSelector', 'BreadthFirstSearch', 'OneTurnLookahead', 'PrunedBFS' and 'TunedTreeTraversal'.

We simulated 600 battles for each combination of policies and used win rate, number of turns and required time as evaluation metrics. 

##### 4.1 Win rate 
Matrix and division by policy
##### 4.2 Number of turns

##### 4.3 Time


#### 5. Conclusions

#### 6 Appendix:
##### 6.1 Team contribution:
- Vito
- Suqi
- Salvatore
- Antonio
- Matilde carried out initial analyses and final assesment <br>

Everyone contibuted on idea development, bug fixing and report writing

##### 6.2 Relationship with the course 
